In [23]:
!pwd

%load_ext autoreload
%autoreload 2

/app


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
import torch
import os
import sys 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
sys.path.append('/app/src')


# これをしないと，sagemathが勝手にintやfloatをinteger型やRealNuber型に変換してしまうしてしまう
preparser(False)

In [25]:
from loader.data import _load_data, SimpleDataCollator
from dataset.tokernizer import set_tokenizer, set_vocab, set_vocab_ver2
from torch.utils.data import DataLoader
from loader.model import load_model




In [26]:
num_variables = 2
field = 'GF7'
max_coefficient = 200
max_degree=20
continous_ids = [2]

params = {'encoding_method': 'standard',
          'd_model': 512,
          'nhead': 8,
          'num_encoder_layers': 6,
          'num_decoder_layers': 6,
          'dim_feedforward': 2048,
          'dropout': 0.1,
          'max_sequence_length': 10000,
          'positional_encoding': 'embedding',
          'regression_weight': 1.0,}

import argparse

params = argparse.Namespace(**params)


trainset = _load_data('./data/shape/shape_n=2_field=GF7/data_GF7_n=2.train.lex.infix')
testset = _load_data('./data/shape/shape_n=2_field=GF7/data_GF7_n=2.test.lex.infix')

print(trainset[0])
print(testset[0])

vocab = set_vocab(num_variables, 
                  field=field, 
                  max_coeff=max_coefficient, 
                  max_degree=max_degree, 
                  continuous_coefficient=False, 
                continuous_exponent=False)

tokenizer = set_tokenizer(vocab)

dc = SimpleDataCollator(tokenizer)


{'input': 'C1 E3 E0 + C1 E2 E0 + C6 E1 E2 + C6 E0 E2 + C1 E0 E1 [SEP] C1 E4 E0 + C2 E3 E0 + C6 E2 E2 + C1 E2 E0 + C5 E1 E2 + C1 E1 E1 + C1 E1 E0 + C6 E0 E2 + C1 E0 E1 + C1 E0 E0', 'target': 'C1 E1 E0 + C1 E0 E0 [SEP] C1 E0 E1', 'input_mask': None, 'target_mask': None}
{'input': 'C1 E3 E0 + C1 E2 E0 + C6 E1 E2 + C6 E0 E2 + C1 E0 E1 [SEP] C1 E4 E0 + C2 E3 E0 + C6 E2 E2 + C1 E2 E0 + C5 E1 E2 + C1 E1 E1 + C1 E1 E0 + C6 E0 E2 + C1 E0 E1 + C1 E0 E0', 'target': 'C1 E1 E0 + C1 E0 E0 [SEP] C1 E0 E1', 'input_mask': None, 'target_mask': None}


In [28]:
loader = DataLoader(trainset, collate_fn=dc, batch_size=16, shuffle=True)

batch = next(iter(loader))

for k, v in batch.items():
    print(k, v.shape)

encoder_input torch.Size([16, 276])
decoder_input torch.Size([16, 32])
encoder_padding_mask torch.Size([16, 276])
decoder_padding_mask torch.Size([16, 32])
labels torch.Size([16, 32])


In [29]:
batch.keys()

dict_keys(['encoder_input', 'decoder_input', 'encoder_padding_mask', 'decoder_padding_mask', 'labels'])

In [41]:
batch['encoder_input'][0]

tensor([ 4, 12, 11, 32,  5, 11, 13, 40,  9, 12, 13, 32,  4, 12, 12, 32,  8, 11,
        15, 32,  6, 11, 14, 32,  9, 11, 13, 32,  4, 11, 12, 38, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
        36, 36, 36, 36, 36, 36, 36, 36, 

In [37]:
batch['labels']

tensor([[ 4, 12, 11, 32,  5, 11, 13, 40,  4, 11, 14, 32,  9, 11, 13, 32,  4, 11,
         12, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  4, 11, 14, 40,  4, 11, 16, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  6, 11, 12, 40,  4, 11, 14, 32,  9, 11, 13, 32,  4, 11,
         12, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  8, 11, 12, 40,  4, 11, 13, 32,  5, 11, 12, 32,  6, 11,
         11, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  5, 11, 13, 32,  9, 11, 11, 40,  4, 11, 14, 38, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 14, 32,  7, 11, 13, 32,  9, 11,
         11, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  6, 11, 12, 32,  9, 11, 11, 40,  4, 11, 15, 32,  6, 11,
         11, 38, 36, 36, 36, 36, 36, 

In [30]:
# パディングトークンは意味を持たないので，このトークンに注意を向けないように教えてあげる
batch['encoder_padding_mask'] # ['PAD']のところがTrue

tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True]])

In [31]:
print(tokenizer.vocab['[PAD]'])
batch['encoder_input'] # 36が　['PAD']のトークンID

36


tensor([[ 4, 12, 11,  ..., 36, 36, 36],
        [ 4, 12, 11,  ..., 36, 36, 36],
        [ 4, 13, 11,  ..., 36, 36, 36],
        ...,
        [ 5, 15, 11,  ..., 36, 36, 36],
        [ 4, 12, 11,  ..., 36, 36, 36],
        [ 5, 15, 11,  ..., 36, 36, 36]])

In [36]:
len(tokenizer.vocab.keys())

43